# Desafio:

Você faz parte da equipe de Analytics de uma grande marca de vestuário com mais de 25 lojas espalhadas em Shoppings de todo o Brasil.

Toda semana você precisa enviar para a diretoria um ranking atualizado com as 25 lojas contendo 3 informações:
- Faturamento de cada Loja
- Quantidade de Produtos Vendidos de cada Loja
- Ticket Médio dos Produto de cada Loja

Além disso, cada loja tem 1 gerente que precisa receber o resumo das informações da loja dele. Por isso, cada gerente deve receber no e-mail:
- Faturamento da sua loja
- Quantidade de Produtos Vendidos da sua loja
- Ticket Médio dos Produto da sua Loja

Esse relatório é sempre enviado como um resumo de todos os dados disponíveis no ano.

# Solução do Desafio:

Para resolver o desafio vamos seguir a seguinte lógica:

- Passo 1 - Importar a base de Dados
- Passo 2 - Visualizar a Base de Dados para ver se precisamos fazer algum tratamento
- Passo 3 - Calcular os indicadores de todas as lojas:
  - Faturamento por Loja
  - Quantidade de Produtos Vendidos por Loja
  - Ticket Médio dos Produto por Loja
- Passo 4 - Configurando o envio de email
- Passo 5 - Calcular os indicadores de cada loja
- Passo 6 - Enviar e-mail para cada loja
- Passo 7 - Enviar e-mail para a diretoria

### Passo 1 - Importando a Base de Dados 

In [3]:
import pandas as pd

In [4]:
#dataframe
df = pd.read_excel('Vendas.xlsx')

### Passo 2 - Visualizando os Dados e verificando a necessidade de fazer tratamento dos Dados

In [5]:
display(df)

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final
0,1,2019-01-01,Iguatemi Esplanada,Sapato Estampa,1,358,358
1,1,2019-01-01,Iguatemi Esplanada,Camiseta,2,180,360
2,1,2019-01-01,Iguatemi Esplanada,Sapato Xadrez,1,368,368
3,2,2019-01-02,Norte Shopping,Relógio,3,200,600
4,2,2019-01-02,Norte Shopping,Chinelo Liso,1,71,71
...,...,...,...,...,...,...,...
100994,69996,2019-12-26,Center Shopping Uberlândia,Short Listrado,2,102,204
100995,69996,2019-12-26,Center Shopping Uberlândia,Mochila,4,270,1080
100996,69996,2019-12-26,Center Shopping Uberlândia,Pulseira Estampa,1,87,87
100997,69997,2019-12-26,Ribeirão Shopping,Camisa Listrado,1,108,108


Observe que não conseguimos ver todos os dados, além disso nossa observação pode nos confundir ou deixar algo passar. Podemos usar o comando a seguir pra ter algumas informações resumidas das **colunas numéricas** 

In [9]:
df.describe()

,Código Venda,Quantidade,Valor Unitário,Valor Final
count,100999.000000,100999.000000,100999.000000,100999.000000
mean,34923.310429,2.165249,191.861018,414.628491
std,20227.910360,1.257710,145.414462,434.093054
min,1.000000,1.000000,30.000000,30.000000
25%,17461.000000,1.000000,100.000000,156.000000
50%,34842.000000,2.000000,155.000000,274.000000
75%,52502.000000,3.000000,248.000000,524.000000
max,69997.000000,5.000000,750.000000,3750.000000


Com isso podemos ver que não há **campos numéricos** sem preenchimento, além disso não temos números negativos, o que é bom pois não faz sentido ter quantidade ou valor negativo. Dessa forma temos consistência em relação a isso.
**Observação:** Aqui verifiquei apenas uma pequena parte dos dados. É necessário verificar mais coisas pra saber o que precisaremos tratar, mas neste projeto, a base já está ok e o objetivo não é ir a fundo nessa parte, sinta-se a vontade pra buscar mais testes e se diveritir!

### Passo 3.1 - Calculando o Faturamento por Loja

O **faturamento** de uma loja é a soma dos valores finais de todos os produtos dela. Para capturar isso, vamos pegar as colunas do **ID Loja** e **Valor Final**, ai agrupamos os ID Lojas repetidos com a função **groupby** e já aproveitamos e somamos os valores finais com a função **sum**.

In [54]:
tabela_faturamento = df[['ID Loja', 'Valor Final']].groupby('ID Loja').sum()
#Dessa forma temos nossa tabela com as lojas e seu faturamento, mas pra ter um ranking vamos ordená-la 
tabela_faturamento = tabela_faturamento.sort_values(by = 'Valor Final', ascending = False)
#Observe que usamos a função sort_values e colocamos dois argumentos, sempre que quiser saber mais sobre uma função
#você deve pesquisar, ir na documentação, essa é uma boa prática que você deve por em prática!

In [55]:
display(tabela_faturamento)

,Valor Final
ID Loja,
Iguatemi Campinas,1762419
Shopping Vila Velha,1731167
Bourbon Shopping SP,1726110
Rio Mar Recife,1722766
Shopping SP Market,1721763
Palladium Shopping Curitiba,1721120
Norte Shopping,1711968
Ribeirão Shopping,1707122
Iguatemi Esplanada,1699681


### Passo 3.2 - Calculando a Quantidade Vendida por Loja

Aqui vamos proceder de maneira parecida com o passo anterior, mas agora queremos o atributo(coluna) **Quantidade**

In [32]:
quantidade = df[['ID Loja', 'Quantidade']].groupby('ID Loja').sum()
quantidade = quantidade.sort_values(by='ID Loja', ascending=False)
display(quantidade)

,Quantidade
ID Loja,
Shopping Vila Velha,9224
Shopping União de Osasco,8730
Shopping SP Market,8927
Shopping Recife,8581
Shopping Morumbi,8508
Shopping Midway Mall,8206
Shopping Iguatemi Fortaleza,8629
Shopping Ibirapuera,8723
Shopping Eldorado,8719


### Passo 3.3 - Calculando o Ticket Médio dos Produtos por Loja

O ticket médio é um indicador de desempenho essencial que fornece o valor médio de vendas por cliente. Para calcular o ticket médio, basta dividir o faturamento total pelo número de vendas do período.
Vamos usar nossas duas tabelas já criadas, que é a do **faturamento** e a da **quantidade**

In [56]:
ticket_medio = tabela_faturamento['Valor Final']/quantidade['Quantidade']

In [57]:
#Vamos ver como ficou nosso ticket medio
ticket_medio

ID Loja
Bourbon Shopping SP                 194.754598
Center Shopping Uberlândia          193.453228
Iguatemi Campinas                   197.248909
Iguatemi Esplanada                  198.098019
Norte Shopping                      189.923231
Novo Shopping Ribeirão Preto        191.775226
Palladium Shopping Curitiba         189.321307
Parque Dom Pedro Shopping           194.519552
Passei das Águas Shopping           191.345324
Ribeirão Shopping                   193.441586
Rio Mar Recife                      194.377299
Rio Mar Shopping Fortaleza          190.044758
Salvador Shopping                   189.323868
Shopping Barra                      191.375666
Shopping Center Interlagos          189.105014
Shopping Center Leste Aricanduva    188.282614
Shopping Eldorado                   189.025232
Shopping Ibirapuera                 187.442394
Shopping Iguatemi Fortaleza         194.092479
Shopping Midway Mall                193.814404
Shopping Morumbi                    186.464974
Shopp

In [46]:
type(ticket_medio)

pandas.core.series.Series

Vendo o **type** observe que temos a estrutura **series** do **pandas**, assim vamos transformar nosso **ticket_medio** em **dataframe**, e dessa forma vamos ter uma apresentação mais elegante. Além disso vamos renomear nossa coluna. 

In [47]:
ticket_medio = ticket_medio.to_frame()
ticket_medio = ticket_medio.rename(columns={0: 'Ticket Médio'})
display(ticket_medio)

,Ticket Médio
ID Loja,
Bourbon Shopping SP,194.754598
Center Shopping Uberlândia,193.453228
Iguatemi Campinas,197.248909
Iguatemi Esplanada,198.098019
Norte Shopping,189.923231
Novo Shopping Ribeirão Preto,191.775226
Palladium Shopping Curitiba,189.321307
Parque Dom Pedro Shopping,194.519552
Passei das Águas Shopping,191.345324


In [49]:
#vamos ordenar nosso Ticket Médio
ticket_medio = ticket_medio.sort_values(by='Ticket Médio', ascending=False)
display(ticket_medio)

,Ticket Médio
ID Loja,
Iguatemi Esplanada,198.098019
Iguatemi Campinas,197.248909
Bourbon Shopping SP,194.754598
Parque Dom Pedro Shopping,194.519552
Rio Mar Recife,194.377299
Shopping Iguatemi Fortaleza,194.092479
Shopping Midway Mall,193.814404
Center Shopping Uberlândia,193.453228
Ribeirão Shopping,193.441586


### Passo 4 - Configurando o envio de email

O código abaixo ajuda a configurar o envio do email, esse é o tipo de configuração que quando você precisar basta pesquisar, a comunidade python é muito ativa e não precisamore reinventar a roda. 

In [65]:
senha = 'hhhh'

In [63]:
# função enviar_email
import smtplib
import email.message

def enviar_email(nome_da_loja, tabela):
  server = smtplib.SMTP('smtp.gmail.com:587')  
  email_content = f'''
  <p>Prezados,</p>
  <p>Segue relatório de Vendas</p>
  {tabela.to_html()}
  <p>Qualquer dúvida estou à disposição</p>'''
  
  msg = email.message.Message()
  msg['Subject'] = f'Relatório de Vendas - Loja: {nome_da_loja}'
  
  
  msg['From'] = 'nathanlubawski1@gmail.com' #seu email
  msg['To'] = 'nathanlubawski@estudante.ufscar.br' #para qual email vai o conteúdo
  password = senha
  msg.add_header('Content-Type', 'text/html')
  msg.set_payload(email_content)
  
  s = smtplib.SMTP('smtp.gmail.com: 587')
  s.starttls()
  # Login Credentials for sending the mail
  s.login(msg['From'], password)
  s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))

### Passo 5 - Calculando Indicadores por Loja 

### Passo 6 - Enviando E-mail para cada Loja

### Passo 7 - Enviando E-mail para a Diretoria

In [ ]:
# email para diretoria

tabela_diretoria = faturamento.join(quantidade).join(ticket_medio)
enviar_email(tabela_diretoria, 'Todas as Lojas')

lojas = df['ID Loja'].unique()

for loja in lojas:
  tabela_loja = df.loc[df['ID Loja'] == loja, ['ID Loja', 'Quantidade', 'Valor Final']]
  resumo_loja = tabela_loja.groupby('ID Loja').sum()
  resumo_loja['Ticket Médio'] = resumo_loja['Valor Final'] / resumo_loja['Quantidade']
  enviar_email(resumo_loja, loja)